# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on MapReduce with Apache Spark** </center>
---
**Profesor**: Pablo Camarillo Ramirez
**Student**: Francisco De Jesus Delgado Carrasco

# Find the PySpark Installation

In [4]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
print("PySpark version:", pyspark.__version__)

ModuleNotFoundError: No module named 'findspark'

# Create SparkSession

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MapReduceExamples") \
    .master("spark://75d0febabe7f:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

ModuleNotFoundError: No module named 'pyspark'

# Example: Count Words
## Create the RDD

In [2]:
words = ["apple", "banana", "apple", "orange",
         "banana", "apple", "orange", "orange"]

words_rdd = sc.parallelize(words)

print("RDD original:")
print(words_rdd.collect())

NameError: name 'sc' is not defined

## Map Phase

In [ ]:
pairs = words_rdd.map(lambda word: (word, 1))
print("Pairs:")
print(pairs.collect())

## Shuffle and Reduce Phases

In [ ]:
word_counts = pairs.reduceByKey(lambda a, b: a + b)

print("Word counts:")
print(word_counts.collect())

## Collect results

In [ ]:
for word, count in word_counts.collect():
    print(f"{word}: {count}")


# Example: Log Analysis
## Create RDD

In [1]:
lines = sc.textFile("/opt/spark/work-dir/data/web_access.log")

NameError: name 'sc' is not defined

## Transform every line in a tuple

In [ ]:
from pcamarillor.spark_utils import parse_line
records = lines.map(parse_line)

## Filter data from yesterday

In [ ]:
from datetime import datetime, timezone, timedelta
from pcamarillor.spark_utils import is_yesterday, to_domain
yesterday = (datetime.now(timezone.utc).date() - timedelta(days=1)).isoformat()
yesterday

In [ ]:
y_records = records.filter(lambda record: is_yesterday(record[2], yesterday))

## Reduce by domain, user

In [ ]:
domain_user_ones = y_records.map(lambda record: ((to_domain(record[1]), record[0]), 1))
unique_domain_user = domain_user_ones.reduceByKey(lambda a, b: 1)

## Count unique users per domain

In [ ]:
domain_ones = unique_domain_user.map(lambda kv: (kv[0][0], 1))
unique_visitors_per_domain = domain_ones.reduceByKey(lambda a, b: a + b)

# Get results

In [ ]:
result = unique_visitors_per_domain.collect()
result

# Close SparkContext

In [ ]:
sc.stop()